In [1]:
import openai
import os
import random
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import nltk
from llama_index.readers.file import UnstructuredReader
from pathlib import Path

# Ensure NLTK data is downloaded
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.find('tokenizers/punkt_tab')

[nltk_data] Downloading package punkt to /Users/boringtao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/boringtao/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


FileSystemPathPointer('/Users/boringtao/nltk_data/tokenizers/punkt_tab')

In [3]:
years = [2022, 2021, 2020, 2019]

loader = UnstructuredReader()
doc_set = {}
all_docs = []
for year in years:
    year_docs = loader.load_data(
        file=Path(f"data/uber/UBER_{year}.html"), split_documents=False
    )
    # insert year metadata into each year
    for d in year_docs:
        d.metadata = {"year": year}
    doc_set[year] = year_docs
    all_docs.extend(year_docs)

In [4]:
print(f"Loaded {len(all_docs)} documents")

Loaded 4 documents


In [6]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.chunk_size = 512
Settings.chunk_overlap = 64
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

index_set = {}
for year in years:
    storage_context = StorageContext.from_defaults()
    cur_index = VectorStoreIndex.from_documents(
        doc_set[year],
        storage_context=storage_context,
    )
    index_set[year] = cur_index
    storage_context.persist(persist_dir=f"storage/uber/{year}")

In [7]:
from llama_index.core import load_index_from_storage

index_set = {}
for year in years:
    storage_context = StorageContext.from_defaults(
        persist_dir=f"storage/uber/{year}"
    )
    cur_index = load_index_from_storage(
        storage_context,
    )
    index_set[year] = cur_index

In [8]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=index_set[year].as_query_engine(),
        metadata=ToolMetadata(
            name=f"vector_index_{year}",
            description=(
                "useful for when you want to answer queries about the"
                f" {year} SEC 10-K for Uber"
            ),
        ),
    )
    for year in years
]

In [9]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
)

In [10]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description=(
            "useful for when you want to answer queries that require analyzing"
            " multiple SEC 10-K documents for Uber"
        ),
    ),
)

In [13]:
tools = individual_query_engine_tools + [query_engine_tool]
print(f"Loaded {len(individual_query_engine_tools)} tools")

Loaded 4 tools


In [14]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [15]:
response = agent.chat("hi, i am bob")
print(str(response))

Added user message to memory: hi, i am bob
Hello Bob! How can I assist you today?


In [16]:
response = agent.chat(
    "What were some of the biggest risk factors in 2020 for Uber?"
)
print(str(response))

Added user message to memory: What were some of the biggest risk factors in 2020 for Uber?
=== Calling Function ===
Calling function: vector_index_2020 with args: {"input":"biggest risk factors"}
Got output: The biggest risk factors include the adverse effects of the COVID-19 pandemic on the business, potential reclassification of Drivers, intense competition in the industries, significant losses and uncertain path to profitability, challenges in maintaining a critical mass of platform users, operational and cultural challenges, negative impact on brand reputation, difficulties in managing growth, safety incidents affecting platform users, financial impacts of the pandemic, and uncertainties related to the pandemic's long-term effects on business operations and financial results.

In 2020, some of the biggest risk factors for Uber included the adverse effects of the COVID-19 pandemic on the business, potential reclassification of Drivers, intense competition in the industries, signific

In [17]:
cross_query_str = (
    "Compare/contrast the risk factors described in the Uber 10-K across"
    " years. Give answer in bullet points."
)

response = agent.chat(cross_query_str)
print(str(response))

Added user message to memory: Compare/contrast the risk factors described in the Uber 10-K across years. Give answer in bullet points.
=== Calling Function ===
Calling function: sub_question_query_engine with args: {"input":"compare and contrast risk factors"}
Generated 4 sub questions.
Got output: Error: Detected nested async. Please use nest_asyncio.apply() to allow nested event loops.Or, use async entry methods like `aquery()`, `aretriever`, `achat`, etc.

=== Calling Function ===
Calling function: vector_index_2019 with args: {"input": "biggest risk factors"}
Got output: The biggest risk factors include higher levels of credit risk and payment fraud, adverse tax consequences, complexities of foreign value added tax systems, difficulties in implementing financial systems across multiple jurisdictions, political and economic instability abroad, public health concerns like coronavirus, reduced protection for intellectual property rights in certain markets, limited influence over minor